In [ ]:





!pip install gTTS googletrans==4.0.0-rc1 ipython ipywidgets pillow


import random
from datetime import datetime
import time
import os
import csv
from gtts import gTTS
from googletrans import Translator
from IPython.display import Audio, display, HTML
import ipywidgets as widgets
from PIL import Image
import io


os.makedirs("excuse_logs", exist_ok=True)
log_file_path = os.path.join("excuse_logs", "daily_excuses.csv")

if not os.path.exists(log_file_path):
    with open(log_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["EmployeeID", "DateTime", "Context", "Excuse", "Proof"])

style = """
    <style>
        .excuse-box {
            border: 2px solid #4CAF50;
            border-radius: 10px;
            padding: 15px;
            background-color: #f9f9f9;
            margin-top: 10px;
            font-family: Arial, sans-serif;
            color: #333;
        }
        .excuse-header {
            color: #4CAF50;
            font-size: 20px;
            font-weight: bold;
        }
        .highlight {
            color: #2196F3;
            font-weight: bold;
        }
    </style>
"""
display(HTML(style))

contexts = {
    "work": [
        "I'm caught up in a critical client meeting and will need a bit more time.",
        "My system just crashed and I lost all progress, trying to recover it now.",
        "There was a sudden internet outage in my area, I’ll be back online shortly."
    ],
    "school": [
        "I was feeling unwell after lunch and couldn’t focus on the assignment.",
        "My assignment file got corrupted right before submission—working on a recovery.",
        "There was a power cut and I couldn’t complete the work on time."
    ],
    "social": [
        "I got stuck in unexpected traffic, really sorry for the delay.",
        "There’s a family emergency I need to attend to—will catch up later.",
        "I’m not feeling well and don’t want to risk spreading anything."
    ],
    "family": [
        "I had to help a relative with a sudden health issue.",
        "There was an urgent family discussion I couldn’t avoid.",
        "I was asked to stay back for a family ritual."
    ],
    "pet": [
        "My dog needed emergency care and I had to rush to the vet.",
        "My cat went missing and I had to look for it around the neighborhood.",
        "I had to rush my pet to the vet due to sudden illness."
    ],
    "weather": [
        "Heavy rain flooded my neighborhood and I couldn’t leave the house.",
        "Storm warnings kept me from traveling for safety reasons.",
        "My area experienced a sudden power outage due to lightning."
    ],
    "health": [
        "I had a severe headache and couldn’t concentrate on anything.",
        "I had a sudden fever and had to rest as per doctor’s advice.",
        "I experienced food poisoning symptoms unexpectedly and had to lie down."
    ],
    "travel": [
        "My flight was delayed due to bad weather conditions.",
        "My bus broke down on the way and I had to wait for assistance.",
        "There was a strike that disrupted transportation in my area."
    ],
    "technical": [
        "My laptop crashed during an update and I lost some progress.",
        "The Wi-Fi router stopped working, waiting for it to reconnect.",
        "The server was down and I couldn't access the system remotely."
    ],
    "mental health": [
        "I needed to take a break for mental well-being and avoid burnout.",
        "I felt overwhelmed and took a moment to calm down before continuing.",
        "I had an anxiety episode and had to step away from everything."
    ],
    "guest": [
        "Unexpected guests arrived and I had to host them temporarily.",
        "Relatives visited without prior notice and I couldn’t excuse myself.",
        "A family friend dropped in urgently and I had to entertain them."
    ],
    "event": [
        "I had to attend a religious ceremony at short notice with family.",
        "A local event blocked my commute and I was stuck in traffic.",
        "I was volunteering at a community function and lost track of time."
    ]
}

proof_templates = [
    "Email screenshot indicating delay: '{}'",
    "Chat log snippet: '{} - seen at 4:47 PM'",
    "GPS log: user last seen at different location when '{}'",
    "Medical note auto-generated for: '{}'",
    "Simulated calendar event overlap: '{}'"
]

def generate_excuse(context):
    excuses = contexts.get(context.lower(), [])
    if not excuses:
        return "No excuses available for this context."
    return random.choice(excuses)

def generate_proof(excuse):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    proof_type = random.choice(proof_templates).format(excuse)
    return f"[Proof - {timestamp}] {proof_type}"

def log_excuse(employee_id, context, excuse, proof):
    with open(log_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([employee_id, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), context, excuse, proof])

def speak_text(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    tts.save("excuse.mp3")
    display(Audio("excuse.mp3", autoplay=True))

def multi_language_support(excuse, language_code):
    translator = Translator()
    try:
        translated = translator.translate(excuse, dest=language_code)
        return f"[{translated.dest.upper()}] {translated.text}"
    except Exception as e:
        return f"Translation error: {str(e)}"

def upload_image():
    upload_widget = widgets.FileUpload(accept='image/*', multiple=False)

    def on_upload_change(change):
        for filename, file_info in upload_widget.value.items():
            image_data = file_info['content']
            image = Image.open(io.BytesIO(image_data))
            display(image)
            with open(os.path.join("excuse_logs", filename), "wb") as f:
                f.write(image_data)
            display(HTML(f"<div class='excuse-box'>✅ Image '{filename}' uploaded and saved.</div>"))

    upload_widget.observe(on_upload_change, names='value')
    display(upload_widget)

context_dropdown = widgets.Dropdown(
    options=list(contexts.keys()),
    description='Context:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
employee_id_input = widgets.Text(
    placeholder='Enter Employee ID',
    description='Employee ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

def on_context_selected(change):
    context = change['new']
    employee_id = employee_id_input.value or "Unknown"
    excuse = generate_excuse(context)
    proof = generate_proof(excuse)
    log_excuse(employee_id, context, excuse, proof)
    apology = f"Sincere apologies for the inconvenience caused. {excuse}"

    display(HTML(f"""
        <div class='excuse-box'>
            <div class='excuse-header'>Generated Excuse:</div>
            {excuse}<br><br>
            <div class='excuse-header'>Generated Proof:</div>
            {proof}<br><br>
            <div class='excuse-header'>Apology:</div>
            {apology}
        </div>
    """))

    speak_text(excuse)

    lang_dropdown = widgets.Dropdown(
        options={"English": "en", "Tamil": "ta", "Kannada": "kn", "Malayalam": "ml", "Hindi": "hi", "Japanese": "ja", "German": "de"},
        description='Translate To:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    def on_language_selected(l_change):
        lang_code = l_change['new']
        translated_excuse = multi_language_support(excuse, lang_code)
        display(HTML(f"<div class='excuse-box'><b>Translated Excuse:</b><br>{translated_excuse}</div>"))
        if lang_code != "en":
            speak_text(translated_excuse.split("] ", 1)[1], lang=lang_code)

    lang_dropdown.observe(on_language_selected, names='value')
    display(lang_dropdown)

    display(HTML("<div class='excuse-box'><b>📷 Upload an Image:</b></div>"))
    upload_image()

display(employee_id_input)
context_dropdown.observe(on_context_selected, names='value')
display(context_dropdown)




  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=af5838f6c309e439c1bcd999cceb51b8efc8edeb7ceab98dcbb4970f7b3e13d1
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe

Text(value='', description='Employee ID:', layout=Layout(width='50%'), placeholder='Enter Employee ID', style=…

Dropdown(description='Context:', layout=Layout(width='50%'), options=('work', 'school', 'social', 'family', 'p…